In [1]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_ta as ta

In [2]:
user = 'alumno'
password = '123456'
host = 'localhost'
port = '5432'
database = 'course-db'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [4]:
df_features = pd.read_sql('SELECT * FROM precios_forex', con=engine)
df_features

,Datetime,Open_EURUSD,High_EURUSD,Low_EURUSD,Close_EURUSD,Open_AUDUSD,High_AUDUSD,Low_AUDUSD,Close_AUDUSD,Open_EURCHF,...,Low_USDCHF,Close_USDCHF,Open_GBPUSD,High_GBPUSD,Low_GBPUSD,Close_GBPUSD,Open_GBPCHF,High_GBPCHF,Low_GBPCHF,Close_GBPCHF
0,2025-06-01 23:00:00+00:00,1.135847,1.136235,1.135718,1.136105,0.645078,0.645186,0.644828,0.645091,0.93346,...,0.82171,0.82195,1.346421,1.346928,1.346366,1.346801,1.10672,1.10700,1.10629,1.10691
1,2025-06-01 23:15:00+00:00,1.136105,1.136105,1.135976,1.136105,0.645078,0.645190,0.644953,0.645190,0.93346,...,0.82185,0.82193,1.346765,1.346820,1.346584,1.346765,1.10688,1.10708,1.10653,1.10684
2,2025-06-01 23:30:00+00:00,1.136235,1.136235,1.135976,1.136105,0.645186,0.645199,0.644995,0.645178,0.93354,...,0.82184,0.82202,1.346856,1.346928,1.346729,1.346928,1.10685,1.10736,1.10658,1.10733
3,2025-06-01 23:45:00+00:00,1.136235,1.136364,1.135460,1.135718,0.645178,0.645340,0.644579,0.644616,0.93366,...,0.82170,0.82220,1.347128,1.347400,1.346692,1.346892,1.10725,1.10748,1.10679,1.10743
4,2025-06-02 00:00:00+00:00,1.135718,1.137398,1.135718,1.137398,0.644608,0.645740,0.644608,0.645599,0.93348,...,0.82108,0.82118,1.347001,1.348909,1.347001,1.348909,1.10743,1.10770,1.10676,1.10759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,2025-06-27 20:15:00+00:00,1.171783,1.171921,1.171783,1.171783,0.653048,0.653249,0.653048,0.653083,0.93639,...,0.79921,0.79937,1.371516,1.371836,1.371460,1.371460,1.09634,1.09666,1.09580,1.09630
1892,2025-06-27 20:30:00+00:00,1.171783,1.171921,1.171783,1.171921,0.653219,0.653471,0.653125,0.653441,0.93633,...,0.79905,0.79909,1.371535,1.371704,1.371385,1.371573,1.09631,1.09633,1.09568,1.09600
1893,2025-06-27 20:45:00+00:00,1.172608,1.172745,1.172058,1.172196,0.653125,0.653479,0.653083,0.653253,0.93624,...,0.79810,0.79836,1.371610,1.372363,1.370877,1.372363,1.09591,1.09601,1.09495,1.09592
1894,2025-06-27 21:00:00+00:00,1.171921,1.171921,1.171921,1.171921,0.653253,0.653253,0.653253,0.653253,0.93574,...,0.79836,0.79836,1.372288,1.372288,1.372288,1.372288,1.09546,1.09546,1.09546,1.09546


In [5]:
# Supongamos que tu DataFrame se llama df
df_indicadores = pd.DataFrame()
df_indicadores['Datetime'] = df_features['Datetime']

# Lista de activos
activos = ['EURUSD', 'AUDUSD', 'EURCHF', 'USDCAD', 'EURJPY', 'EURGBP',
           'GBPJPY', 'NZDUSD', 'USDJPY', 'USDCHF', 'GBPUSD', 'GBPCHF']

# Para cada activo generamos los indicadores
for activo in activos:
    close = df_features[f'Close_{activo}']
    high = df_features[f'High_{activo}']
    low = df_features[f'Low_{activo}']

    # RSI (14)
    df_indicadores[f'RSI_{activo}'] = ta.rsi(close, length=14)

    # MACD
    macd = ta.macd(close)
    df_indicadores[f'MACD_{activo}'] = macd[f'MACD_12_26_9']
    df_indicadores[f'MACD_signal_{activo}'] = macd[f'MACDs_12_26_9']
    df_indicadores[f'MACD_hist_{activo}'] = macd[f'MACDh_12_26_9']

    # Bandas de Bollinger
    bbands = ta.bbands(close, length=20)
    df_indicadores[f'BBL_{activo}'] = bbands[f'BBL_20_2.0']
    df_indicadores[f'BBM_{activo}'] = bbands[f'BBM_20_2.0']
    df_indicadores[f'BBU_{activo}'] = bbands[f'BBU_20_2.0']

    # EMA (20)
    df_indicadores[f'EMA_{activo}'] = ta.ema(close, length=20)

    # ATR (14)
    df_indicadores[f'ATR_{activo}'] = ta.atr(high=high, low=low, close=close, length=14)

    # ROC (Rate of Change)
    df_indicadores[f'ROC_{activo}'] = ta.roc(close, length=10)

df_indicadores

C:\Users\haide\AppData\Local\Temp\ipykernel_12296\3536329382.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_indicadores[f'RSI_{activo}'] = ta.rsi(close, length=14)
C:\Users\haide\AppData\Local\Temp\ipykernel_12296\3536329382.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_indicadores[f'MACD_{activo}'] = macd[f'MACD_12_26_9']
C:\Users\haide\AppData\Local\Temp\ipykernel_12296\3536329382.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

,Datetime,RSI_EURUSD,MACD_EURUSD,MACD_signal_EURUSD,MACD_hist_EURUSD,BBL_EURUSD,BBM_EURUSD,BBU_EURUSD,EMA_EURUSD,ATR_EURUSD,...,RSI_GBPCHF,MACD_GBPCHF,MACD_signal_GBPCHF,MACD_hist_GBPCHF,BBL_GBPCHF,BBM_GBPCHF,BBU_GBPCHF,EMA_GBPCHF,ATR_GBPCHF,ROC_GBPCHF
0,2025-06-01 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-01 23:15:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-01 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-06-01 23:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-06-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,2025-06-27 20:15:00+00:00,53.401247,-0.000278,-0.000366,0.000088,1.169125,1.171455,1.173785,1.171311,0.000751,...,54.533298,-0.000058,-0.000219,0.000160,1.094833,1.095702,1.096572,1.095865,0.001003,0.078505
1892,2025-06-27 20:30:00+00:00,54.593115,-0.000195,-0.000331,0.000137,1.169185,1.171393,1.173602,1.171369,0.000707,...,50.614031,-0.000044,-0.000184,0.000140,1.094838,1.095716,1.096595,1.095877,0.000978,0.108690
1893,2025-06-27 20:45:00+00:00,56.964924,-0.000105,-0.000286,0.000181,1.169213,1.171366,1.173519,1.171448,0.000715,...,49.590813,-0.000039,-0.000155,0.000116,1.094838,1.095715,1.096593,1.095881,0.000984,0.086758
1894,2025-06-27 21:00:00+00:00,53.931156,-0.000056,-0.000240,0.000184,1.169336,1.171290,1.173245,1.171493,0.000684,...,44.073820,-0.000071,-0.000138,0.000067,1.094814,1.095688,1.096563,1.095841,0.000947,0.014617


In [9]:
df_indicadores.fillna(0, inplace=True)

In [13]:
df_indicadores

,Datetime,RSI_EURUSD,MACD_EURUSD,MACD_signal_EURUSD,MACD_hist_EURUSD,BBL_EURUSD,BBM_EURUSD,BBU_EURUSD,EMA_EURUSD,ATR_EURUSD,...,RSI_GBPCHF,MACD_GBPCHF,MACD_signal_GBPCHF,MACD_hist_GBPCHF,BBL_GBPCHF,BBM_GBPCHF,BBU_GBPCHF,EMA_GBPCHF,ATR_GBPCHF,ROC_GBPCHF
0,2025-06-01 23:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2025-06-01 23:15:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2025-06-01 23:30:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2025-06-01 23:45:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2025-06-02 00:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,2025-06-27 20:15:00+00:00,53.401247,-0.000278,-0.000366,0.000088,1.169125,1.171455,1.173785,1.171311,0.000751,...,54.533298,-0.000058,-0.000219,0.000160,1.094833,1.095702,1.096572,1.095865,0.001003,0.078505
1892,2025-06-27 20:30:00+00:00,54.593115,-0.000195,-0.000331,0.000137,1.169185,1.171393,1.173602,1.171369,0.000707,...,50.614031,-0.000044,-0.000184,0.000140,1.094838,1.095716,1.096595,1.095877,0.000978,0.108690
1893,2025-06-27 20:45:00+00:00,56.964924,-0.000105,-0.000286,0.000181,1.169213,1.171366,1.173519,1.171448,0.000715,...,49.590813,-0.000039,-0.000155,0.000116,1.094838,1.095715,1.096593,1.095881,0.000984,0.086758
1894,2025-06-27 21:00:00+00:00,53.931156,-0.000056,-0.000240,0.000184,1.169336,1.171290,1.173245,1.171493,0.000684,...,44.073820,-0.000071,-0.000138,0.000067,1.094814,1.095688,1.096563,1.095841,0.000947,0.014617


In [14]:
# Especifica el nombre de la tabla que deseas crear o actualizar
nombre_tabla = 'features_forex'

# Enviar el DataFrame
df_indicadores.to_sql(nombre_tabla, engine, if_exists='replace', index=False)

6